In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import scipy.stats as st
def one_run(n=20):
    x = np.random.standard_normal(n)
    xbar = x.mean()
    s_uncorr = np.sqrt(np.mean((x - xbar)**2))
    s_corr   = np.sqrt(np.sum((x - xbar)**2)/(n-1))
    tcrit = st.t.ppf(0.975, df=n-1)
    ci_uncorr = (xbar - tcrit*s_uncorr/np.sqrt(n), xbar + tcrit*s_uncorr/np.sqrt(n))
    ci_corr   = (xbar - tcrit*s_corr/np.sqrt(n),   xbar + tcrit*s_corr/np.sqrt(n))
    contain_uncorr = (0 >= ci_uncorr[0]) and (0 <= ci_uncorr[1])
    contain_corr   = (0 >= ci_corr[0]) and (0 <= ci_corr[1])
    return contain_uncorr, contain_corr

In [7]:
contain_uncorr, contain_corr = one_run(20)
print("Uncorrected covers 0?", contain_uncorr)
print("Corrected covers 0?", contain_corr)

Uncorrected covers 0? True
Corrected covers 0? True


In [8]:
trials = 100_000
count_uncorr = 0
count_corr = 0

for i in range(trials):
    cu, cc = one_run(20)
    count_uncorr += cu
    count_corr   += cc

coverage_uncorr = count_uncorr / trials
coverage_corr   = count_corr / trials

print(f"After {trials} trials:")
print(f"Uncorrected coverage = {coverage_uncorr:.5f}")
print(f"Corrected coverage   = {coverage_corr:.5f}")

After 100000 trials:
Uncorrected coverage = 0.94472
Corrected coverage   = 0.95069


In [ ]:
# A properly calibrated 95% CI should contain the true mean ≈ 0.95 of the time. 
# The Bessel-corrected estimator is closest to this target; 
# the uncorrected estimator systematically under-covers because it underestimates variability.
# **Conclusion.**
# The Bessel-corrected variance (divide by n−1) is less biased for estimating the population variance 
# Out of 100 000 (or 1 000 000) simulations, the corrected CI contained the true mean ≈ 95 % of the time,
# while the uncorrected CI did so ≈ 94 %.


In [9]:
coverage_uncorr = 0.94
coverage_corr   = 0.95

In [11]:
#Scenario	Sampling pattern	n	Tests per year
#1	Daily check	    10	365
#2	Every minute	10	365 × 24 × 60 = 525 600
#3	Every 2 minutes	20	262 800False positives per year=Tests per year×(1−coverage)
#False positives per year=Tests per year×(1−coverage)

In [12]:
coverage_n10_uncorr = 0.94   
coverage_n10_corr   = 0.95
coverage_n20_uncorr = 0.945
coverage_n20_corr   = 0.95

def expected_false_positives(coverage, tests):
    return tests * (1 - coverage)

# Tests per year
days_per_year = 365
minutes_per_year = 365 * 24 * 60
tests_daily = days_per_year
tests_minutely = minutes_per_year
tests_every2min = minutes_per_year / 2

# Scenario 1 – Daily, n=10
fp_daily_unc = expected_false_positives(coverage_n10_uncorr, tests_daily)
fp_daily_cor = expected_false_positives(coverage_n10_corr, tests_daily)

# Scenario 2 – Every minute, n=10
fp_min_unc = expected_false_positives(coverage_n10_uncorr, tests_minutely)
fp_min_cor = expected_false_positives(coverage_n10_corr, tests_minutely)

# Scenario 3 – Every 2 minutes, n=20
fp_2min_unc = expected_false_positives(coverage_n20_uncorr, tests_every2min)
fp_2min_cor = expected_false_positives(coverage_n20_corr, tests_every2min)

print("=== Part 2 – Expected False Positives per Year ===")
print(f"Daily (n=10):    Uncorrected={fp_daily_unc:.2f}, Corrected={fp_daily_cor:.2f}, Extra={fp_daily_unc - fp_daily_cor:.2f}")
print(f"Every minute (n=10): Uncorrected={fp_min_unc:.2f}, Corrected={fp_min_cor:.2f}, Extra={fp_min_unc - fp_min_cor:.2f}")
print(f"Every 2 minutes (n=20): Uncorrected={fp_2min_unc:.2f}, Corrected={fp_2min_cor:.2f}, Extra={fp_2min_unc - fp_2min_cor:.2f}")
#In practical application of Bessel's correction, I got assisstance from AI.


=== Part 2 – Expected False Positives per Year ===
Daily (n=10):    Uncorrected=21.90, Corrected=18.25, Extra=3.65
Every minute (n=10): Uncorrected=31536.00, Corrected=26280.00, Extra=5256.00
Every 2 minutes (n=20): Uncorrected=14454.00, Corrected=13140.00, Extra=1314.00


In [ ]:
# Daily: If everything is fine, you’d still expect around 18 – 22 false alerts/year, and 3–4 extra if you skip Bessel’s correction.
# Every minute: Roughly 26 000 – 31 000 alerts/year, with about 5 000 extra from ignoring Bessel’s correction.
# Every 2 minutes: Roughly 13 000 – 14 000 alerts/year, with about 1 400 extra from ignoring correction.